## Import

In [1]:
import sklearn

import random
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms


import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 
import wandb

/home/insung/fall_detection/fall_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorrt

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [4]:
CFG = {
    'VIDEO_LENGTH':50,
    'EPOCHS':100,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':42,
    'TRAIN_DIR':'./data_new/train',
    'TEST_DIR':'./data_new/test'
}

In [5]:
wandb.init(project="thermal_fall_new", config=CFG)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: is-jang (is-jang-pusan-national-university). Use `wandb login --relogin` to force relogin


wandb: 500 encountered ({"errors":[{"message":"context deadline exceeded","path":["project"]}],"data":{"project":null}}), retrying request
wandb: Network error resolved after 0:01:01.290248, resuming normal operation.


## Fixed RandomSeed

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Load

## Train / Validation Split

## CustomDataset

In [7]:

class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.classes = os.listdir(root_dir)
        self.num_frames = 50

        self.video_paths = []
        self.labels = []

        for label, cls in enumerate(self.classes):
            cls_path = os.path.join(root_dir, cls)
            video_files = os.listdir(cls_path)
            for video_file in video_files:
                video_path = os.path.join(cls_path, video_file)
                self.video_paths.append(video_path)
                # # print(video_path + str(label)) # no 0 good 1 fall 2
                # if label == 0:
                #     print("bin")
                #     label = 1
                self.labels.append(label)

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        frames = sorted(os.listdir(video_path))[:self.num_frames]

        video_frames = []
        for frame in frames:
            img_path = os.path.join(video_path, frame)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (64, 64))
            img = img / 255.0

            angle = random.uniform(-10, 10)
            M = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), angle, 1)
            img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
            
            video_frames.append(img)
        
        video_frames = np.stack(video_frames)
        video_frames = np.expand_dims(video_frames, axis=0)
        video_frames = torch.FloatTensor(video_frames)

        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long)

        return video_frames, label


In [8]:

# Dataset 및 타겟 정의
train_dataset = CustomDataset(CFG['TRAIN_DIR'])
targets = train_dataset.labels

# StratifiedShuffleSplit 사용
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=CFG['SEED'])

# StratifiedShuffleSplit은 인덱스를 반환하므로 이를 활용해 train/val 인덱스 분리
for train_idx, val_idx in stratified_split.split(train_dataset, targets):
    train_dataset_split = Subset(train_dataset, train_idx)
    val_dataset_split = Subset(train_dataset, val_idx)

# DataLoader 설정
train_loader = DataLoader(train_dataset_split, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
valid_loader = DataLoader(val_dataset_split, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


## Model Define

In [9]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=2):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Conv3d(1, 8, (1, 3, 3)), 
            nn.ReLU(),
            nn.BatchNorm3d(8),
            nn.MaxPool3d((1, 2, 2)),  
            nn.Conv3d(8, 32, (1, 3, 3)),  
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.MaxPool3d((1, 2, 2)),  
            nn.Conv3d(32, 64, (1, 3, 3)),  
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.AdaptiveAvgPool3d((1, 1, 1)),  
        )
        self.classifier = nn.Linear(64, num_classes)  

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

In [10]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        
        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta
        

    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
            
        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

## Train

In [11]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        wandb.log({
            'epoch': epoch,
            'train_loss': _train_loss,
            'val_loss': _val_loss,
            'val_f1': _val_score
        })

        es(_val_loss)

        if es.early_stop:
            print("Early Stopping")
            break
    torch.save(model.state_dict(),  'small.pt')
        

            

In [12]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='weighted')
    return _val_loss, _val_score

## Run!!

In [13]:
model = BaseModel()
es = EarlyStopping(patience=10, delta=0.0, mode='min', verbose=True)
# model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
# torch.quantization.prepare_qat(model, inplace=True)
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])


In [14]:
train(model, optimizer, train_loader, valid_loader, device)

100%|██████████| 5/5 [00:00<00:00, 32.12it/s]


Epoch [1], Train Loss : [0.63821] Val Loss : [0.62506] Val F1 : [0.68056]
[EarlyStopping] (Update) Best Score: 0.62506


100%|██████████| 5/5 [00:00<00:00, 42.87it/s]


Epoch [2], Train Loss : [0.61364] Val Loss : [0.71180] Val F1 : [0.51393]
[EarlyStopping] (Patience) 1/10, Best: 0.62506, Current: 0.71180, Delta: 0.08674


100%|██████████| 5/5 [00:00<00:00, 43.36it/s]


...

Epoch [24], Train Loss : [0.37212] Val Loss : [0.48207] Val F1 : [0.82430]
[EarlyStopping] (Patience) 8/10, Best: 0.46727, Current: 0.48207, Delta: 0.01480


100%|██████████| 5/5 [00:00<00:00, 42.04it/s]


Epoch [25], Train Loss : [0.37855] Val Loss : [0.64673] Val F1 : [0.64363]
[EarlyStopping] (Patience) 9/10, Best: 0.46727, Current: 0.64673, Delta: 0.17946


100%|██████████| 5/5 [00:00<00:00, 40.39it/s]

Epoch [26], Train Loss : [0.38038] Val Loss : [0.63588] Val F1 : [0.64081]
[EarlyStopping] (Patience) 10/10, Best: 0.46727, Current: 0.63588, Delta: 0.16860
[EarlyStop Triggered] Best Score: 0.46727
Early Stopping


In [15]:
model.load_state_dict(torch.load('small.pt'))
model.eval()

BaseModel(
  (feature_extract): Sequential(
    (0): Conv3d(1, 8, kernel_size=(1, 3, 3), stride=(1, 1, 1))
    (1): ReLU()
    (2): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv3d(8, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1))
    (5): ReLU()
    (6): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (8): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
    (9): ReLU()
    (10): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  )
  (classifier): Linear(in_features=64, out_features=2, bias=True)
)

## Inference

In [16]:
test_dataset = CustomDataset(CFG["TEST_DIR"])
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [17]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    true_labels = []
    with torch.no_grad():
        for videos, labels in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()


    f1 = f1_score(true_labels, preds, average='weighted') 
    print(f"f1 score = [{f1}]")

    return preds, f1

In [18]:
preds, f1 = inference(model, test_loader, device)

100%|██████████| 6/6 [00:00<00:00, 35.04it/s]

f1 score = [0.819047619047619]


wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
